In [70]:
# imports 
import pandas as pd
import seaborn as sns
import os

In [71]:
# load the dataset
base_path = os.path.join("..", "data")
energy_path = os.path.join(base_path, "energy_dataset.csv")
weather_path = os.path.join(base_path, "weather_features.csv")

df = pd.read_csv(energy_path)
df_weather = pd.read_csv(weather_path)
df_weather = df_weather[df_weather["city_name"] == "Valencia"]
df_weather.drop(columns=["city_name"], inplace=True)
df

,time,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,...,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
0,2015-01-01 00:00:00+01:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,...,196.0,0.0,6378.0,17.0,NaN,6436.0,26118.0,25385.0,50.10,65.41
1,2015-01-01 01:00:00+01:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,...,195.0,0.0,5890.0,16.0,NaN,5856.0,24934.0,24382.0,48.10,64.92
2,2015-01-01 02:00:00+01:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,...,196.0,0.0,5461.0,8.0,NaN,5454.0,23515.0,22734.0,47.33,64.48
3,2015-01-01 03:00:00+01:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,...,191.0,0.0,5238.0,2.0,NaN,5151.0,22642.0,21286.0,42.27,59.32
4,2015-01-01 04:00:00+01:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,...,189.0,0.0,4935.0,9.0,NaN,4861.0,21785.0,20264.0,38.41,56.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,2018-12-31 19:00:00+01:00,297.0,0.0,0.0,7634.0,2628.0,178.0,0.0,0.0,0.0,...,277.0,0.0,3113.0,96.0,NaN,3253.0,30619.0,30653.0,68.85,77.02
35060,2018-12-31 20:00:00+01:00,296.0,0.0,0.0,7241.0,2566.0,174.0,0.0,0.0,0.0,...,280.0,0.0,3288.0,51.0,NaN,3353.0,29932.0,29735.0,68.40,76.16
35061,2018-12-31 21:00:00+01:00,292.0,0.0,0.0,7025.0,2422.0,168.0,0.0,0.0,0.0,...,286.0,0.0,3503.0,36.0,NaN,3404.0,27903.0,28071.0,66.88,74.30
35062,2018-12-31 22:00:00+01:00,293.0,0.0,0.0,6562.0,2293.0,163.0,0.0,0.0,0.0,...,287.0,0.0,3586.0,29.0,NaN,3273.0,25450.0,25801.0,63.93,69.89


In [72]:
df["generation_other"] = (
    df["generation biomass"]
    + df["generation geothermal"]
    + df["generation waste"]
    + df.get("generation other", 0)  # in case there's an existing "other" column
)

df["generation_coal"] = (
    df["generation fossil brown coal/lignite"]
    + df["generation fossil hard coal"]
    + df["generation fossil coal-derived gas"]
)

df["generation_oil"] = (
    df["generation fossil oil"]
    + df.get("generation fossil oil shale", 0)
    + df.get("generation fossil peat", 0)
)

df["generation_natural_gas"] = df["generation fossil gas"]

df["generation_solar"] = df["generation solar"]

df["forecast_solar"] = df["forecast solar day ahead"]

df["generation_wind"] = (
    df["generation wind onshore"]
    + df["generation wind offshore"]
)

df["forecast_wind"] = (
    df["forecast wind onshore day ahead"].fillna(0)
    + df["forecast wind offshore eday ahead"].fillna(0)
)

df = df[[
    "time",
    "generation_other",
    "generation_coal",
    "generation_oil",
    "generation_natural_gas",
    "generation_solar",
    "forecast_solar",
    "generation_wind",
    "forecast_wind",
    "total load forecast",
    "total load actual",
    "price day ahead",
    "price actual"
]]
df = df.rename(columns={
    "total load forecast":"total_load_forecast",
    "total load actual":"total_load_actual",
    "price day ahead":"price_day_ahead",
    "price actual":"price_actual"}
    )

df = df.copy()
df


,time,generation_other,generation_coal,generation_oil,generation_natural_gas,generation_solar,forecast_solar,generation_wind,forecast_wind,total_load_forecast,total_load_actual,price_day_ahead,price_actual
0,2015-01-01 00:00:00+01:00,686.0,5150.0,162.0,4844.0,49.0,17.0,6378.0,6436.0,26118.0,25385.0,50.10,65.41
1,2015-01-01 01:00:00+01:00,687.0,5083.0,158.0,5196.0,50.0,16.0,5890.0,5856.0,24934.0,24382.0,48.10,64.92
2,2015-01-01 02:00:00+01:00,687.0,4904.0,157.0,4857.0,50.0,8.0,5461.0,5454.0,23515.0,22734.0,47.33,64.48
3,2015-01-01 03:00:00+01:00,672.0,4385.0,160.0,4314.0,50.0,2.0,5238.0,5151.0,22642.0,21286.0,42.27,59.32
4,2015-01-01 04:00:00+01:00,660.0,4027.0,156.0,4130.0,42.0,9.0,4935.0,4861.0,21785.0,20264.0,38.41,56.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,2018-12-31 19:00:00+01:00,637.0,2628.0,178.0,7634.0,85.0,96.0,3113.0,3253.0,30619.0,30653.0,68.85,77.02
35060,2018-12-31 20:00:00+01:00,638.0,2566.0,174.0,7241.0,33.0,51.0,3288.0,3353.0,29932.0,29735.0,68.40,76.16
35061,2018-12-31 21:00:00+01:00,639.0,2422.0,168.0,7025.0,31.0,36.0,3503.0,3404.0,27903.0,28071.0,66.88,74.30
35062,2018-12-31 22:00:00+01:00,641.0,2293.0,163.0,6562.0,31.0,29.0,3586.0,3273.0,25450.0,25801.0,63.93,69.89


In [73]:
# Now let's make the data into something we can make predictions on:

df["price_actual_lag1"] = df["price_actual"].shift(24)  # yesterday’s same-hour price
df["generation_wind_lag1"] = df["generation_wind"].shift(24)
df["generation_solar_lag1"] = df["generation_solar"].shift(24)

df["time"] = pd.to_datetime(df["time"], errors="coerce", utc=True)
df["hour"] = df["time"].dt.hour
df["day"] = df["time"].dt.day
df["month"] = df["time"].dt.month
df["year"] = df["time"].dt.year
df["dayofweek"] = df["time"].dt.dayofweek    # Monday = 0, Sunday = 6
df["is_weekend"] = df["dayofweek"] >= 5      # True/False


# Target = tomorrow’s actual price
df["target_price_tomorrow"] = df["price_actual"].shift(-24)

# Drop last 24 hours (no target available)
df = df.dropna(subset=["target_price_tomorrow","price_actual_lag1"])

# Features (everything we would know before tomorrow)

df

,time,generation_other,generation_coal,generation_oil,generation_natural_gas,generation_solar,forecast_solar,generation_wind,forecast_wind,total_load_forecast,...,price_actual_lag1,generation_wind_lag1,generation_solar_lag1,hour,day,month,year,dayofweek,is_weekend,target_price_tomorrow
24,2015-01-01 23:00:00+00:00,582.0,1291.0,193.0,3189.0,54.0,5.0,13232.0,13329.0,27309.0,...,65.41,6378.0,49.0,23,1,1,2015,3,False,66.82
25,2015-01-02 00:00:00+00:00,580.0,1190.0,192.0,2902.0,54.0,35.0,12760.0,12718.0,25397.0,...,64.92,5890.0,50.0,0,2,1,2015,4,False,63.35
26,2015-01-02 01:00:00+00:00,569.0,1023.0,189.0,2772.0,54.0,43.0,12174.0,12375.0,23640.0,...,64.48,5461.0,50.0,1,2,1,2015,4,False,58.79
27,2015-01-02 02:00:00+00:00,578.0,1016.0,188.0,2936.0,53.0,32.0,11358.0,11524.0,22638.0,...,59.32,5238.0,50.0,2,2,1,2015,4,False,57.44
28,2015-01-02 03:00:00+00:00,579.0,1103.0,189.0,2893.0,53.0,31.0,11199.0,11310.0,22238.0,...,56.04,4935.0,42.0,3,2,1,2015,4,False,55.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35035,2018-12-30 18:00:00+00:00,633.0,1510.0,228.0,5730.0,31.0,70.0,5129.0,5305.0,29201.0,...,71.01,7354.0,31.0,18,30,12,2018,6,True,77.02
35036,2018-12-30 19:00:00+00:00,643.0,1556.0,221.0,5683.0,31.0,50.0,5406.0,5302.0,30140.0,...,70.72,7807.0,31.0,19,30,12,2018,6,True,76.16
35037,2018-12-30 20:00:00+00:00,646.0,1616.0,216.0,5859.0,31.0,29.0,5193.0,5258.0,30378.0,...,70.34,7710.0,23.0,20,30,12,2018,6,True,74.30
35038,2018-12-30 21:00:00+00:00,656.0,1610.0,215.0,5660.0,31.0,40.0,4912.0,5112.0,29418.0,...,68.65,7411.0,15.0,21,30,12,2018,6,True,69.89


In [ ]:
target_feature = "target_price_tomorrow"
X = df.drop(columns=[target_feature,"time","forecast_solar","forecast_wind"],axis=1)
y = df[target_feature]
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import TimeSeriesSplit
X

,generation_other,generation_coal,generation_oil,generation_natural_gas,generation_solar,forecast_solar,generation_wind,forecast_wind,total_load_forecast,total_load_actual,...,price_actual,price_actual_lag1,generation_wind_lag1,generation_solar_lag1,hour,day,month,year,dayofweek,is_weekend
24,582.0,1291.0,193.0,3189.0,54.0,5.0,13232.0,13329.0,27309.0,27070.0,...,64.02,65.41,6378.0,49.0,23,1,1,2015,3,False
25,580.0,1190.0,192.0,2902.0,54.0,35.0,12760.0,12718.0,25397.0,24935.0,...,58.46,64.92,5890.0,50.0,0,2,1,2015,4,False
26,569.0,1023.0,189.0,2772.0,54.0,43.0,12174.0,12375.0,23640.0,23214.0,...,54.70,64.48,5461.0,50.0,1,2,1,2015,4,False
27,578.0,1016.0,188.0,2936.0,53.0,32.0,11358.0,11524.0,22638.0,22540.0,...,54.91,59.32,5238.0,50.0,2,2,1,2015,4,False
28,579.0,1103.0,189.0,2893.0,53.0,31.0,11199.0,11310.0,22238.0,22096.0,...,53.07,56.04,4935.0,42.0,3,2,1,2015,4,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35035,633.0,1510.0,228.0,5730.0,31.0,70.0,5129.0,5305.0,29201.0,29221.0,...,72.37,71.01,7354.0,31.0,18,30,12,2018,6,True
35036,643.0,1556.0,221.0,5683.0,31.0,50.0,5406.0,5302.0,30140.0,30042.0,...,73.35,70.72,7807.0,31.0,19,30,12,2018,6,True
35037,646.0,1616.0,216.0,5859.0,31.0,29.0,5193.0,5258.0,30378.0,30229.0,...,73.64,70.34,7710.0,23.0,20,30,12,2018,6,True
35038,656.0,1610.0,215.0,5660.0,31.0,40.0,4912.0,5112.0,29418.0,29145.0,...,71.92,68.65,7411.0,15.0,21,30,12,2018,6,True


In [75]:
tscv = TimeSeriesSplit(n_splits=20)

model = LinearRegression()

for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    print(X_train)
    print(f"TRAIN indices: {train_index.min()}-{train_index.max()}, TEST indices: {test_index.min()}-{test_index.max()}")
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)

    # You would typically train your model here:
    # model.fit(X_train, y_train)
    # And then evaluate on the test set:
    # predictions = model.predict(X_test)




# Kan ikke bruke vanlig train test split pga data leakage
# Vi skal bruke walk forward validation, fordi det funker best for tids serie data
# https://medium.com/@ahmedfahad04/understanding-walk-forward-validation-in-time-series-analysis-a-practical-guide-ea3814015abf

# Vi har noen valg også
# Valg 1: Vi kan ha en modell som tar time by time data og returnerer hva som skjer om en dag
# Valg 2: eller så kan vi ha en modell som sier hva som skjer om en time

# For å da predikere en hva som skjer iløpet av en hel dag fremover
# Valg 1 så blir prediksjonen vår klokken 18 imorgen bli basert på klokken 18 igår
# valg 2 så blri prediksjonen vår klokken 01:00 basert på 24:00 igår

# Ulemper:
# I valg 1 så har vi ikke konteksten til hva som har skjedd mellom klokken 18 igår og 18 idag
# når vi skal predikere. Dette kan være viktig data. Fordel: Enkelt å predikere en dag frem i tid på
# I valg 2 så har vi problem så fort vi skal gå mer enn en time frem i tid. Ettersom all dataen vår ikke blir riktig
# Vi kommer til å mangle *all* tidsbasert data. Så da er vi avhengig av å predikere alle våre temporale data.
# Så vi på predikere pris på olje,naturgass,kull,vær,elektrisitet produksjon osv...
# Det man kan ha er en model for å predikere 1,2,3,4,5,...,24 timer frem i tid hver for seg
# Altså 24 forskjellige modeller

#model = LinearRegression()
#model.fit(X_train,y_train)
#y_pred = y.predict(X_test)
#accuracy = accuracy_score(y_test,y_pred)
#print(accuracy)


      generation_other  generation_coal  generation_oil  \
24               582.0           1291.0           193.0   
25               580.0           1190.0           192.0   
26               569.0           1023.0           189.0   
27               578.0           1016.0           188.0   
28               579.0           1103.0           189.0   
...                ...              ...             ...   
1695             720.0           8363.0           431.0   
1696             722.0           8343.0           437.0   
1697             715.0           8426.0           437.0   
1698             713.0           8532.0           438.0   
1699             720.0           8420.0           441.0   

      generation_natural_gas  generation_solar  forecast_solar  \
24                    3189.0              54.0             5.0   
25                    2902.0              54.0            35.0   
26                    2772.0              54.0            43.0   
27                    2936.

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values